In [1]:
!pip install langchain langgraph langsmith langchain-groq langchain_community langchain-tavily chromadb

INFO: pip is looking at multiple versions of langchain-groq to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of langgraph-prebuilt to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langgraph-prebuilt to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/war

In [2]:
groq_api_key="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"
tavily_api_key="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"

In [3]:
import uuid
from typing import TypedDict, Annotated
from langchain_core.messages import AIMessage, HumanMessage
from langchain_tavily import TavilySearch
from langgraph.graph import StateGraph, START
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
import chromadb
from chromadb.utils import embedding_functions
from langchain.tools import tool

In [4]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.3-70b-versatile", groq_api_key=groq_api_key)

In [5]:
class State(TypedDict):
  messages: Annotated[list, add_messages]
  conversation_id: str

tool = TavilySearch(max_results=2, tavily_api_key=tavily_api_key)

def multiply(a: int, b: int) -> int:
    """this tool is responsible for Multiplication"""
    return a * b

tools = [tool, multiply]
llm_with_tools = llm.bind_tools(tools)

In [6]:
#initialize chromadb client
chroma_client = chromadb.Client()
embedding_function = embedding_functions.DefaultEmbeddingFunction()
collection = chroma_client.create_collection(
    name="conversation_history_1",
    embedding_function=embedding_function
)

def store_in_chroma(state: State):
  """ Store Conversation Messages in ChromaDB. """
  conversation_id = state.get("conversation_id", str(uuid.uuid4()))
  messages = state["messages"]

  # Convert messages to text for embedding
  message_texts = [f"{msg.type}: {msg.content}" for msg in messages]

  #Store in ChromaDB
  collection.add(
      documents=message_texts,
      metadatas=[{"conversation_id": conversation_id, "index": i} for i in range(len(messages))],
      ids=[f"{conversation_id}_{i}" for i in range(len(messages))]
  )
  return conversation_id

def retrieve_context(state: State) -> str:
  """ Retrieve relevant context from chromaDB. """
  conversation_id = state.get("conversation_id", "")

  last_message = state["messages"][-1].content if state["messages"] else ""

  if not conversation_id or not last_message:
    return ""

  #Query ChromaDB for relevant context
  results=collection.query(
      query_texts=[last_message],
      n_results=3,
      where={"conversation_id": conversation_id}
  )

  #combine relevant context
  context = "\n".join(doc for doc in results['documents'][0] if doc)
  return context

In [7]:
def tool_calling_llm(state: State):
  #Retrieve relevant context
  context = retrieve_context(state)

  # Add context to the prompt if available
  if context:
    prompt = f"Context from previous conversation:\n{context}\n\nCurrent Message:\n{state["messages"]}"
  else:
    prompt = state["messages"]

  # Store conversation in ChromaDB and get conversation_id
  conversation_id = store_in_chroma(state)

  #update state with conversation_id
  state["conversation_id"] = conversation_id

  return {
      "messages": [llm_with_tools.invoke(prompt)],
      "conversation_id": conversation_id
  }

In [8]:
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import START, END

#initialize memory
memory = MemorySaver()

# Build AI workflow with graph
builder = StateGraph(State)

builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode(tools))

# Add Edges
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    tools_condition
)

builder.add_edge("tool_calling_llm", END)

#compile the graph
graph = builder.compile(checkpointer=memory)

In [9]:
config={"configurable":{"thread_id":"3"}}
response = graph.invoke({
    "messages": [HumanMessage(content="please tell me about myself within two lines")],
    "conversation_id": str(uuid.uuid4())
}, config)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 33.6MiB/s]


In [12]:
response["messages"][-1]

AIMessage(content="I don't have any information about you. \nYou are an anonymous user.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 1853, 'total_tokens': 1870, 'completion_time': 0.068935233, 'prompt_time': 0.118792443, 'queue_time': 0.099902874, 'total_time': 0.187727676}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_fe13ff5dcd', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--c4637de9-b127-44da-a660-2f10eac9e6aa-0', usage_metadata={'input_tokens': 1853, 'output_tokens': 17, 'total_tokens': 1870})